In [14]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    confusion_matrix, classification_report

from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve

from sklearn.model_selection import learning_curve

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.inspection import permutation_importance
#import warnings
#warnings.simplefilter(action="ignore")

# Baseline model

In [7]:
immo = pd.read_csv("immo.csv")

In [8]:
# define X, y
y = immo["ocean_proximity_number"]
X = immo.drop("ocean_proximity_number", axis=1)

In [9]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.43564292201894383

---Now we can see the score of my baseline: 0.43---

In [11]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.4428294573643411

---We can see a little improvement with the dataset : immo_with_outliers_and_imput and immo_with_outliers---

---for now i'll keep the outliers---

# First iteration

In [11]:
immo_with_imput = pd.read_csv("immo_with_imput.csv")

In [34]:
def Lineareg(data):
    y = data["median_house_value"]
    X = data.drop("median_house_value", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    print(f'model score: {model.score(X_test, y_test)}')

    K = []
    total_time = []
    score = []
    for k in range(2,20):
        cross_val_results = cross_validate(model, X, y, cv=k)
        total_time.append(sum(cross_val_results['fit_time'])+sum(cross_val_results['score_time']))
        K.append(k)
        score.append(cross_val_results['test_score'].mean())
    best_cv = pd.DataFrame({'K': K,'score': score})
    cross = best_cv.query(f'score=={max(score)}')['K'].item()
    cv_results = cross_validate(model,X, y,cv=cross, scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_error'))
    r2 = cv_results['test_r2'].mean()
    rmse = cv_results['test_neg_root_mean_squared_error'].mean()
    print(f'r2: {r2}')
    print(f'rmse: {rmse}')

In [35]:
Lineareg(immo_with_imput)

model score: 0.6058961194280346
r2: 0.6016233887270823
rmse: -56747.678579654465


# Second iteration

In [36]:
immo_with_outliers = pd.read_csv("immo_with_outliers.csv")
immo_with_outliers_and_imput = pd.read_csv("immo_with_outliers_and_imput.csv")

In [37]:
Lineareg(immo_with_outliers)

model score: 0.6324224669228723
r2: 0.6353530950356457
rmse: -69539.75736337178


In [38]:
Lineareg(immo_with_outliers_and_imput)

model score: 0.6324224669228723
r2: 0.6353530950356457
rmse: -69539.75736337178
